In [1]:
def train(in_c, in_gamma, in_kernel):
    import seaborn as sns
    from sklearn.svm import SVC
    from sklearn.model_selection import train_test_split
    from sklearn import metrics
    from sklearn.datasets import load_iris
    
    import mlflow
    import mlflow.sklearn

    #Split data into features and target
    iris = load_iris()
    X = iris.data
    y = iris.target

    #Split data into test and train data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)
    
    # Set default values if no n is provided
    if float(in_c) is None:
        c = 1
    else:
        c = float(in_c)
        
    if float(in_gamma) is None:
        gamma = 1
    else:
        gamma = float(in_gamma)
        
    if in_kernel is None:
        kernel = 'rbf'
    else:
        kernel = in_kernel
    
    with mlflow.start_run():
        
        #Create SVC model
        svc_model = SVC(C=in_c, kernel=in_kernel, gamma=in_gamma)

        #Train the model
        svc_model.fit(X_train,y_train)

        #Predict
        predictions = svc_model.predict(X_test)
        
        #Compare prediction to actual data and calculate the accuracy
        accuracy = metrics.accuracy_score(y_test, predictions)
        precision =  metrics.precision_score(y_test, predictions, average = 'micro')
        recall = metrics.recall_score(y_true = y_test, y_pred = predictions, average = 'weighted')
        f1_score = metrics.f1_score(y_true = y_test, y_pred = predictions, average = 'micro')
        
         #Log parameter, metrics, and model to MLflow
        mlflow.log_param("c", c)
        mlflow.log_param("gamma", gamma)
        mlflow.log_param("kernel", kernel)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric('f1', f1_score)
        
        #Log the model
        mlflow.sklearn.log_model(svc_model, "model")



In [2]:
c_list = [0.1,1, 10, 100]
gamma_list = [1,0.1,0.01,0.001]
kernel_list = ['rbf', 'poly', 'sigmoid']

In [3]:
for x in c_list:
    for y in gamma_list:
        for z in kernel_list:
            train(x, y, z)